In [17]:
import numpy as np
import os
import train_dense
import train_cnn

In [18]:
def load_mnist():
    data_dir = '../data'

    fd = open(os.path.join(data_dir, 'train-images-idx3-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    trX = loaded[16:].reshape((60000, 28, 28, 1)).astype(np.float)

    fd = open(os.path.join(data_dir, 'train-labels-idx1-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    trY = loaded[8:].reshape((60000)).astype(np.int)

    fd = open(os.path.join(data_dir, 't10k-images-idx3-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    teX = loaded[16:].reshape((10000, 28, 28, 1)).astype(np.float)

    fd = open(os.path.join(data_dir, 't10k-labels-idx1-ubyte'))
    loaded = np.fromfile(file=fd, dtype=np.uint8)
    teY = loaded[8:].reshape((10000)).astype(np.int)

    trY = np.asarray(trY)
    teY = np.asarray(teY)

    perm = np.random.permutation(trY.shape[0])
    trX = trX[perm]
    trY = trY[perm]

    perm = np.random.permutation(teY.shape[0])
    teX = teX[perm]
    teY = teY[perm]

    return trX, trY, teX, teY

In [19]:
trainX, trainY, testX, testY = load_mnist()

In [20]:
# Imports
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib

tf.logging.set_verbosity(tf.logging.INFO)

In [21]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features, [-1, 28, 28, 1])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == learn.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=10)

    loss = None
    train_op = None

    # Calculate Loss (for both TRAIN and EVAL modes)
    if mode != learn.ModeKeys.INFER:
        onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
        loss = tf.losses.softmax_cross_entropy(
            onehot_labels=onehot_labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == learn.ModeKeys.TRAIN:
        train_op = tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.contrib.framework.get_global_step(),
            learning_rate=0.001,
            optimizer="SGD")

    # Generate Predictions
    predictions = {
      "classes": tf.argmax(
          input=logits, axis=1),
      "probabilities": tf.nn.softmax(
          logits, name="softmax_tensor")
    }

    # Return a ModelFnOps object
    return model_fn_lib.ModelFnOps(
      mode=mode, predictions=predictions, loss=loss, train_op=train_op)

In [33]:
# def main(unused_argv):
    # Load training and eval data
# mnist = learn.datasets.load_dataset("mnist")
# trainX, trainY, testX, testY = load_mnist()
train_data = np.asarray(trainX, dtype=np.float32) # Returns np.array
train_labels = np.asarray(trainY, dtype = np.float32)
eval_data = np.asarray(testX, dtype = np.float32) # Returns np.array
eval_labels = np.asarray(testY, dtype = np.float32)

# Estimator
mnist_classifier = learn.Estimator(
  model_fn=cnn_model_fn, model_dir="./tmp")

# Logging hook
# tensors_to_log = {"probabilities": "softmax_tensor"}
# logging_hook = tf.train.LoggingTensorHook(
#   tensors=tensors_to_log, every_n_iter=50)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_environment': 'local', '_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, '_tf_random_seed': None, '_task_type': None, '_save_checkpoints_secs': 600, '_evaluation_master': '', '_save_summary_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000018D83DBE358>, '_keep_checkpoint_max': 5, '_num_ps_replicas': 0, '_save_checkpoints_steps': None, '_is_chief': True}


In [6]:
trainX.shape

(60000, 28, 28, 1)

In [35]:
# mnist_classifier.fit(
#     x=train_data,
#     y=train_labels,
#     batch_size=100,
#     steps=2000,
#     monitors=[logging_hook])

mnist_classifier.fit(
    x=train_data,
    y=train_labels,
    batch_size=100,
    steps=2000)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


D:\Program Files\Anaconda3\envs\tensorflow_tuts\lib\site-packages\tensorflow\python\util\deprecation.py:247: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 4001 into ./tmp\model.ckpt.
INFO:tensorflow:loss = 0.0476271, step = 4001
INFO:tensorflow:global_step/sec: 38.4686
INFO:tensorflow:loss = 0.0400276, step = 4101
INFO:tensorflow:global_step/sec: 38.9957
INFO:tensorflow:loss = 0.00456867, step = 4201
INFO:tensorflow:global_step/sec: 39.4518
INFO:tensorflow:loss = 0.058118, step = 4301
INFO:tensorflow:global_step/sec: 39.4752
INFO:tensorflow:loss = 0.0585079, step = 4401
INFO:tensorflow:global_step/sec: 38.6848
INFO:tensorflow:loss = 0.0183061, step = 4501
INFO:tensorflow:global_step/sec: 39.0111
INFO:tensorflow:loss = 0.0131014, step = 4601
INFO:tensorflow:global_step/sec: 38.9574
INFO:tensorflow:loss = 0.20054, step = 4701
INFO:tensorflow:global_step/sec: 39.8696
INFO:tensorflow:loss = 0.0480168, step = 4801
INFO:tensorflow:global_step/sec: 40.1264
INFO:tensorflow:loss = 0.0261494, step = 4901
INFO:tensorflow:global_step/sec: 39.0339
INFO:tensorflow:loss 

Estimator(params=None)

In [36]:
# Configure the accuracy metric for evaluation
metrics = {
    "accuracy":
        learn.MetricSpec(
            metric_fn=tf.metrics.accuracy, prediction_key="classes"),
}

# Evaluate the model and print results
eval_results = mnist_classifier.evaluate(
    x=eval_data, y=eval_labels, metrics=metrics)
print(eval_results)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


D:\Program Files\Anaconda3\envs\tensorflow_tuts\lib\site-packages\tensorflow\python\util\deprecation.py:247: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


INFO:tensorflow:Starting evaluation at 2017-03-18-18:48:38
INFO:tensorflow:Finished evaluation at 2017-03-18-18:48:39
INFO:tensorflow:Saving dict for global step 6000: accuracy = 0.9856, global_step = 6000, loss = 0.040561
{'global_step': 6000, 'loss': 0.040561013, 'accuracy': 0.98559999}


{'params': None}